In [1]:
import os
import sys
import glob
import sqlite3
from pathlib import Path
import ee
import geemap
import geopandas as gpd
import pandas as pd
import xee
import xarray as xr
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs
import pyet
import spei

sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

from app.utils.gee import run_with_adaptive_buffer, extract_points_to_csv


ee.Authenticate()
ee.Initialize(
    project = 'drought-monitoring-iran',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [2]:
DATABASE_PATH = "../database/database.db"

conn = sqlite3.connect(DATABASE_PATH)

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()


# Download Data

In [3]:
DATASETS = {
    "NDVI_Terra": {
        "image_collection_id": "MODIS/061/MOD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "NDVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "NDVI_Terra_MOD13A3",
        "output_path": f"../output/GEE/VI/NDVI_Terra_MOD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "EVI_Terra": {
        "image_collection_id": "MODIS/061/MOD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "EVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "EVI_Terra_MOD13A3",
        "output_path": f"../output/GEE/VI/EVI_Terra_MOD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "NDVI_Aqua": {
        "image_collection_id": "MODIS/061/MYD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "NDVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "NDVI_Aqua_MYD13A3",
        "output_path": f"../output/GEE/VI/NDVI_Aqua_MYD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "EVI_Aqua": {
        "image_collection_id": "MODIS/061/MYD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "EVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "EVI_Aqua_MYD13A3",
        "output_path": f"../output/GEE/VI/EVI_Aqua_MYD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Day_Terra": {
        "image_collection_id": "MODIS/061/MOD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Day",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Day_Terra_MOD21C3",
        "output_path": f"../output/GEE/VI/LST_Day_Terra_MOD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Night_Terra": {
        "image_collection_id": "MODIS/061/MOD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Night",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Night_Terra_MOD21C3",
        "output_path": f"../output/GEE/VI/LST_Night_Terra_MOD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Day_Aqua": {
        "image_collection_id": "MODIS/061/MYD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Day",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Day_Aqua_MYD21C3",
        "output_path": f"../output/GEE/VI/LST_Day_Aqua_MYD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Night_Aqua": {
        "image_collection_id": "MODIS/061/MYD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Night",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Night_Aqua_MYD21C3",
        "output_path": f"../output/GEE/VI/LST_Night_Aqua_MYD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
}

In [4]:
for name, config in DATASETS.items():
    print(f"Product: {name}")
    df = run_with_adaptive_buffer(
        config=config,
        base_points_geojson=config["points_geojson"],
        buffer_list_m=[0, 1500, 2000, 2500, 5000, 7500, 10000],
    )  
    out_path = Path(config["output_path"])
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False)
    print(f"Final merged result saved to {out_path}")

Product: NDVI_Terra
Nominal scale for NDVI_Terra_MOD13A3: 926.6254330558331

=== Try 1 with buffer_m = 0 m ===
Using scale: 926.6254330558331, buffer_m: 0
Stations still all-NaN after buffer 0 m: set()
All stations have at least some non-NaN values. Stopping.
Final merged result saved to ..\output\GEE\VI\NDVI_Terra_MOD13A3.csv
Product: EVI_Terra
Nominal scale for EVI_Terra_MOD13A3: 926.6254330558331

=== Try 1 with buffer_m = 0 m ===
Using scale: 926.6254330558331, buffer_m: 0
Stations still all-NaN after buffer 0 m: set()
All stations have at least some non-NaN values. Stopping.
Final merged result saved to ..\output\GEE\VI\EVI_Terra_MOD13A3.csv
Product: NDVI_Aqua
Nominal scale for NDVI_Aqua_MYD13A3: 926.6254330558331

=== Try 1 with buffer_m = 0 m ===
Using scale: 926.6254330558331, buffer_m: 0
Stations still all-NaN after buffer 0 m: set()
All stations have at least some non-NaN values. Stopping.
Final merged result saved to ..\output\GEE\VI\NDVI_Aqua_MYD13A3.csv
Product: EVI_Aqua
N

# Join *.CSV Files

In [ ]:
folder = "../output/GEE/VI/"

results = pd.DataFrame()

for filepath in glob.glob(os.path.join(folder, "*.csv")):
    df = pd.read_csv(filepath)
    
    filename = os.path.basename(filepath)
    model_name, _ = os.path.splitext(filename)    
    model_name = model_name.split("-")[0]
    df = df[['region_id', 'Region', 'St_ID', 'St_Name', 'date', model_name]]

    
    if results.empty:
        results = df.copy()
    else:
        results = results.merge(df, on=['region_id', 'Region', 'St_ID', 'St_Name', 'date'], how='outer')
    
results.rename(
    columns={
        "Region": "region_name",
        "St_ID": "station_id",
        "St_Name": "station_name"
    },
    inplace=True
)

results = results[results["date"].apply(lambda x: x.split("-")[2] == "01")]
results['date'] = pd.to_datetime(results['date'])
results['date'] = results['date'] + pd.offsets.MonthEnd(0)

ndvi_columns = [col for col in results.columns if 'NDVI' in col]
results['NDVI'] = results[ndvi_columns].mean(axis=1)

evi_columns = [col for col in results.columns if 'EVI' in col]
results['EVI'] = results[evi_columns].mean(axis=1)

lst_day_columns = [col for col in results.columns if 'LST_Day' in col]
results['LST_Day'] = results[lst_day_columns].mean(axis=1)

lst_night_columns = [col for col in results.columns if 'LST_Night' in col]
results['LST_Night'] = results[lst_night_columns].mean(axis=1)

results['LST'] = results[['LST_Day', 'LST_Night']].mean(axis=1)

results = results.sort_values(by=["region_name", "region_id", "station_name", "station_id", "date"]).reset_index(drop=True)

results

,region_id,region_name,station_id,station_name,date,EVI_Aqua_MYD13A3,EVI_Terra_MOD13A3,LST_Day_Aqua_MYD21C3,LST_Day_Terra_MOD21C3,LST_Night_Aqua_MYD21C3,LST_Night_Terra_MOD21C3,NDVI_Aqua_MYD13A3,NDVI_Terra_MOD13A3,NDVI,EVI,LST_Day,LST_Night,LST
0,MASA,Mazandaran,99361,Alasht,2000-02-29,NaN,0.1194,NaN,5.4748,NaN,5.4344,NaN,0.2218,0.22180,0.11940,5.4748,5.4344,5.4546
1,MASA,Mazandaran,99361,Alasht,2000-03-31,NaN,0.1371,NaN,5.5564,NaN,5.5128,NaN,0.2558,0.25580,0.13710,5.5564,5.5128,5.5346
2,MASA,Mazandaran,99361,Alasht,2000-04-30,NaN,0.1942,NaN,5.9716,NaN,5.6512,NaN,0.3867,0.38670,0.19420,5.9716,5.6512,5.8114
3,MASA,Mazandaran,99361,Alasht,2000-05-31,NaN,0.2595,NaN,6.0724,NaN,5.6824,NaN,0.4515,0.45150,0.25950,6.0724,5.6824,5.8774
4,MASA,Mazandaran,99361,Alasht,2000-06-30,NaN,0.2217,NaN,6.1268,NaN,5.7572,NaN,0.3620,0.36200,0.22170,6.1268,5.7572,5.9420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.3734,0.3853,6.2444,6.0232,5.7000,5.7144,0.5634,0.5523,0.55785,0.37935,6.1338,5.7072,5.9205
4646,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,0.2985,0.2895,6.2796,6.0832,5.7392,5.7308,0.5440,0.5536,0.54880,0.29400,6.1814,5.7350,5.9582
4647,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,0.1651,0.2486,6.1492,5.9760,5.6604,5.6540,0.4590,0.4897,0.47435,0.20685,6.0626,5.6572,5.8599
4648,MASA,Mazandaran,40735,Siahbisheh,2025-10-31,0.1323,0.1565,6.0180,5.8836,5.6256,5.5732,0.3729,0.3600,0.36645,0.14440,5.9508,5.5994,5.7751


In [4]:
conn = sqlite3.connect(DATABASE_PATH)

vi_dataset.to_sql('gee_indices_monthly', conn, if_exists='replace', index=False)
conn.commit()

conn.close()